In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall nltk -y
!pip install texthero transformers

Uninstalling nltk-3.2.5:
  Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 2.0MB 23.0MB/s 
     |████████████████████████████████| 245kB 43.1MB/s 
     |████████████████████████████████| 1.4MB 54.9MB/s 
     |████████████████████████████████| 3.2MB 60.5MB/s 
     |████████████████████████████████| 890kB 48.3MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434675 sha256=087cc67ae67d45657031b7f0e405a041b5bf32f1381f59966fcbc996b0526cfc
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=639eab365494f7a7cf62886f7d3cad10e6025b07fdf4cec9936c24000d2c63a4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built nltk sacremoses


In [3]:
import os

import numpy as np
import pandas as pd

import dill
from sklearn.feature_extraction.text import TfidfVectorizer
import gc
from tqdm import tqdm
import time

import gensim
from gensim.models import Word2Vec
import hashlib
import nltk

import texthero as hero
from texthero import preprocessing

import random
import torch
import transformers

from pathlib import Path
from transformers import BertTokenizer
from tqdm import tqdm

def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(42)

INPUT_DIR = '/content/drive/MyDrive/citation_prediction/input/'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


データの読み込み

In [4]:
all_title_abstract = pd.read_feather(os.path.join(INPUT_DIR, 'all_title_abstract_df.feather'))
all_title_abstract_exists_cites = all_title_abstract[all_title_abstract['cites'].notnull()].reset_index(drop=True)
all_title_abstract_df = pd.concat([all_title_abstract_exists_cites, all_title_abstract.iloc[851524:, :]]).reset_index(drop=True)
print(all_title_abstract_df.shape)
all_title_abstract_df.head()

(74201, 5)


,title,abstract,cites,clean_title,clean_abstract
0,Spectropolarimetric Constraints on the Nature ...,While it is well recognized that interstella...,7.0,spectropolarimetric constraints nature inte...,well recognized interstellar grains made ...
1,A joint analysis of Planck and BICEP2 B modes ...,We analyze BICEP2 and Planck data using a mo...,188.0,joint analysis planck bicep2 b modes includ...,analyze bicep2 planck data using model inc...
2,Molecular movie of ultrafast coherent rotation...,Recording molecular movies on ultrafast time...,8.0,molecular movie ultrafast coherent rotational...,recording molecular movies ultrafast timescal...
3,Early X-ray/UV Line Signatures of GRB Progenit...,We calculate the X-ray/UV spectral line sign...,2.0,early x ray uv line signatures grb progenitor...,calculate x ray uv spectral line signatures ...
4,Neutrino oscillations with three flavors in ma...,"In this paper, we discuss the evolution oper...",14.0,neutrino oscillations three flavors matter ...,paper discuss evolution operator transit...


In [5]:
tqdm.pandas()

class BertSequenceVectorizer:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model_name = '/content/drive/MyDrive/citation_prediction/scibert_scivocab_uncased' # modelを保存しているpathを指定
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128
        print(self.device)

    def vectorize(self, sentence : str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():
          target = pooled_out[0].cpu().detach().numpy()
          return target
        else:
          target = pooled_out[0].detach().numpy()
          return target

BSV = BertSequenceVectorizer()
for col in ['clean_title', 'clean_abstract']: 
    all_title_abstract_bert \
    = all_title_abstract_df[col].fillna('nan').progress_apply(lambda x: BSV.vectorize(x))
    pd.DataFrame(all_title_abstract_bert.tolist(),columns=[f'{col}_scibert_{i}' for i in range(768)]).to_feather(os.path.join(INPUT_DIR, 'scibert_{0}_cls_df.feather'.format(col)))

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version

  0%|          | 2/74201 [00:00<1:24:55, 14.56it/s]

cuda


100%|██████████| 74201/74201 [18:45<00:00, 65.91it/s]


In [18]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model_name = '/content/drive/MyDrive/citation_prediction/scibert_scivocab_uncased' # modelを保存しているpathを指定
# tokenizer = BertTokenizer.from_pretrained(model_name)
# bert_model = transformers.BertModel.from_pretrained(model_name)
# bert_model = bert_model.to(device)
# max_len = 128
# print(device)

# sentence = all_title_abstract_df['clean_title'].iloc[0]
# inp = tokenizer.encode(sentence)
# len_inp = len(inp)

# if len_inp >= max_len:
#   inputs = inp[:max_len]
#   masks = [1] * max_len
# else:
#   inputs = inp + [0] * (max_len - len_inp)
#   masks = [1] * len_inp + [0] * (max_len - len_inp)

# inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(device)
# masks_tensor = torch.tensor([masks], dtype=torch.long).to(device)

# bert_out = bert_model(inputs_tensor, masks_tensor)
# seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

cuda


In [13]:
# sentence

'spectropolarimetric constraints   nature  interstellar grains'

In [12]:
# input_ids = torch.tensor(inp)
# tokenizer.convert_ids_to_tokens(inp)

['[CLS]',
 'spectro',
 '##polar',
 '##imetric',
 'constraints',
 'nature',
 'inters',
 '##tel',
 '##lar',
 'grains',
 '[SEP]']